# price data to mongodb

In [1]:
import requests
import json
import pymongo
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from pprint import pprint
from dateutil.relativedelta import relativedelta
from pymongo import MongoClient
%matplotlib inline

### 確定路徑、讀檔

In [2]:
import os
os.getcwd()

'd:\\coding\\workingspacePython\\most-fintech'

In [3]:
# read data
csv_path = 'C:\\Users\kbjiyu-NB\Desktop\mostfintech_db\price_csv\coindesk_bpi_USD_ohlc_data_2010_07_01_2017_09_17.csv'
df = pd.read_csv(csv_path)

### 清洗、轉換格式

In [4]:
# remove last 2  [ not required data]
df = df.drop(df.index[[-1,-2]])
df.tail()

,Date,Open,High,Low,Close
2669,2017-09-17 12:45:00,3587.98,3591.58,3585.55,3591.63
2670,2017-09-17 13:00:00,3591.63,3594.68,3584.25,3583.97
2671,2017-09-17 13:15:00,3583.97,3620.09,3583.97,3620.34
2672,2017-09-17 13:30:00,3620.34,3663.69,3620.34,3662.14
2673,2017-09-17 13:45:00,3662.14,3663.23,3657.38,3660.83


In [5]:
# date format to datetime
for index, row in df.iterrows():
    x = datetime.datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S')
    df.set_value(index, 'Date', x)

In [6]:
print(type(df['Date'][0]))
df.head()

<class 'datetime.datetime'>


,Date,Open,High,Low,Close
0,2010-07-18 00:00:00,0.05,0.09,0.06,0.09
1,2010-07-19 00:00:00,0.09,0.09,0.08,0.08
2,2010-07-20 00:00:00,0.08,0.08,0.07,0.07
3,2010-07-21 00:00:00,0.07,0.08,0.07,0.08
4,2010-07-22 00:00:00,0.08,0.08,0.05,0.05


### 選擇時間區段

In [7]:
# remove last ones without entire one-date data
start_date = '2017-01-01 00:00:00'
end_date  = '2017-09-17 00:00:00'

start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date  = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

df = df[(df.Date < end_date)]
df.tail()

,Date,Open,High,Low,Close
2613,2017-09-12 00:00:00,4188.84,4368.22,4067.41,4148.27
2614,2017-09-13 00:00:00,4148.27,4152.07,3766.36,3874.26
2615,2017-09-14 00:00:00,3874.26,3923.98,3209.44,3226.41
2616,2017-09-15 00:00:00,3226.41,3761.18,2951.15,3686.90
2617,2017-09-16 00:00:00,3686.90,3874.63,3551.76,3678.74


### 存成list，用於輸入至db

In [8]:
tt = []
for index, row in df.iterrows():
    temp = {
        "Date": row['Date'],
        "Open": row['Open'],
        "High": row['High'],
        "Low": row['Low'],
        "Close": row['Close']
    }
    tt.append(temp)

In [9]:
tt

[{'Close': 0.09,
  'Date': datetime.datetime(2010, 7, 18, 0, 0),
  'High': 0.09,
  'Low': 0.06,
  'Open': 0.05},
 {'Close': 0.08,
  'Date': datetime.datetime(2010, 7, 19, 0, 0),
  'High': 0.09,
  'Low': 0.08,
  'Open': 0.09},
 {'Close': 0.07,
  'Date': datetime.datetime(2010, 7, 20, 0, 0),
  'High': 0.08,
  'Low': 0.07,
  'Open': 0.08},
 {'Close': 0.08,
  'Date': datetime.datetime(2010, 7, 21, 0, 0),
  'High': 0.08,
  'Low': 0.07,
  'Open': 0.07},
 {'Close': 0.05,
  'Date': datetime.datetime(2010, 7, 22, 0, 0),
  'High': 0.08,
  'Low': 0.05,
  'Open': 0.08},
 {'Close': 0.06,
  'Date': datetime.datetime(2010, 7, 23, 0, 0),
  'High': 0.07,
  'Low': 0.05,
  'Open': 0.05},
 {'Close': 0.05,
  'Date': datetime.datetime(2010, 7, 24, 0, 0),
  'High': 0.06,
  'Low': 0.05,
  'Open': 0.06},
 {'Close': 0.05,
  'Date': datetime.datetime(2010, 7, 25, 0, 0),
  'High': 0.06,
  'Low': 0.05,
  'Open': 0.05},
 {'Close': 0.06,
  'Date': datetime.datetime(2010, 7, 26, 0, 0),
  'High': 0.06,
  'Low': 0.05,


### 存入MongoDB

In [10]:
client = MongoClient('mongodb://localhost:27017/')
db = client.bitcoinprice_db
# db.CoindeskDailyCollection.insert_many(price)
for item in tt:
    db.CoindeskDailyCollection.insert_one(item)

# '###################################################################'

### date format to pandas.tslib.Timestamp

In [ ]:
# date format to pandas.tslib.Timestamp
# df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d %H:%M:%S")
# df.head()
# print(type(df['Date'][0])) # str

### iterrow() with df.set_value()

In [ ]:
# divide 1000, make it perfect
# for index, row in df.iterrows():
#     x = float(row['Date']/1000)
#     df.set_value(index, 'Date', x)

# 轉換Date timestamp to datetime.datime
# for index, row in df.iterrows():
#     x = datetime.datetime.fromtimestamp(row['Date'])
#     df.set_value(index, 'Date', x)

### 轉json 筆記

In [ ]:
# price = json.loads(df.T.to_json()).values()

### Timestamp轉換筆記

In [ ]:
# timestamp = 1279411200000
import datetime
timestamp = 1279411200.000
print(type(timestamp))
value = datetime.datetime.fromtimestamp(timestamp)
print(value.strftime('%Y-%m-%d %H:%M:%S'))

### Datetime筆記

<pre>
datetime轉 str 
> x.strftime('%Y-%m-%d')
> str(datimeObject)

str to datetime
> datetime.datetime.strptime(df['Date'], '%Y-%m-%d %H:%M:%S')
> df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d %H:%M:%S")
Python格式化日期时间的函数为datetime.datetime.strftime()；由字符串转为日期型的函数为：datetime.datetime.strptime()，两个函数都涉及日期时间的格式化字符串，列举如下：

%a Abbreviated weekday name 
%A Full weekday name 
%b Abbreviated month name 
%B Full month name 
%c Date and time representation appropriate for locale 
%d Day of month as decimal number (01 - 31) 
%H Hour in 24-hour format (00 - 23) 
%I Hour in 12-hour format (01 - 12) 
%j Day of year as decimal number (001 - 366) 
%m Month as decimal number (01 - 12) 
%M Minute as decimal number (00 - 59) 
%p Current locale's A.M./P.M. indicator for 12-hour clock 
%S Second as decimal number (00 - 59) 
%U Week of year as decimal number, with Sunday as first day of week (00 - 51) 
%w Weekday as decimal number (0 - 6; Sunday is 0) 
%W Week of year as decimal number, with Monday as first day of week (00 - 51) 
%x Date representation for current locale 
%X Time representation for current locale 
%y Year without century, as decimal number (00 - 99) 
%Y Year with century, as decimal number 
%z, %Z Time-zone name or abbreviation; no characters if time zone is unknown 
%% Percent sign

举一个例子：

ebay中时间格式为‘Sep-21-09 16:34’

则通过下面代码将这个字符串转换成datetime

>>> c = datetime.datetime.strptime('Sep-21-09 16:34','%b-%d-%y %H:%M');
>>> c
datetime.datetime(2009, 9, 21, 16, 34)

又如：datetime转换成字符串

>>> datetime.datetime.now().strftime('%b-%d-%y %H:%M:%S');
'Sep-22-09 16:48:08'


</pre>